In [1]:
from captum.attr import IntegratedGradients
import random
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# a random model
class Net(nn.Module):
    def __init__(self, num_hidden):
        super().__init__()
        self.lin1 = nn.Linear(50, num_hidden)
        self.lin2 = nn.Linear(num_hidden, num_hidden)
        self.lin3 = nn.Linear(num_hidden, 2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input):
        lin1 = F.relu(self.lin1(input))
        lin2 = F.relu(self.lin2(lin1))
        lin3 = self.lin3(lin2)
        return self.softmax(lin3)

In [3]:
def completeness_test(model, inp, baseline, attribution, target_class_index, tolerance = 1e-3):
    f_inp = model(inp)
    f_baseline = model(baseline)
    
    diff = (f_inp[0][target_class_index] - f_baseline[0][target_class_index]).sum()
    attr_sum = attribution.sum()
    print(f"sum of attributions {attr_sum.item()}")
    print(f"difference of network output at input as baseline {diff.item()}")
    print(f"approximation error {torch.abs((diff-attr_sum)).item()}")
    
    assert torch.abs(attr_sum - diff) <= tolerance, "failed to pass completness axiom of integrated gradients"
    
    print(f"completness test: passed")

In [4]:
# create input and baseline
inp = torch.arange(0.0, 1.0, 0.02, requires_grad=True).unsqueeze(0)
baseline = torch.zeros_like(inp, requires_grad=True)
model = Net(20)
target_class_index = 1

Testing IG as implemented in captum:

In [5]:
# applying integrated gradients on the SoftmaxModel and input data point
ig = IntegratedGradients(model)
attributions, approximation_error = ig.attribute(inp, target=target_class_index, return_convergence_delta=True)

completeness_test(model, inp, baseline, attributions, target_class_index)

sum of attributions 0.0026382685909034052
difference of network output at input as baseline 0.002300351858139038
approximation error 0.00033791673276436715
completness test: passed


IG as implemented locally (sometimes passes depending on seed, but fails more often than not):

In [17]:
import os
os.chdir('..')
from scripts.gradients import run_integrated_jacobian_scanvi

# Helmholtz method
model = Net(20)
# num_in = 50
inp = torch.arange(0.0, 1.0, 0.02).unsqueeze(0)
baseline = torch.zeros_like(inp)

batches = [{"X":inp, "batch":1}]
# target_class_index = 1
# applying integrated gradients on the SoftmaxModel and input data point
ig_helmholtz = run_integrated_jacobian_scanvi(model, batches, n_steps=10000)
completeness_test(model, inp, baseline, ig_helmholtz[..., target_class_index], target_class_index)

sum of attributions -0.002332361415028572
difference of network output at input as baseline -0.0044172704219818115
approximation error 0.0020849090069532394


AssertionError: failed to pass completness axiom of integrated gradients